### Data Loading & Preprocess

#### _Loading Data_

In [1]:
import os, sys, pickle
import numpy as np
from itertools import chain

cwd = os.getcwd()
print(cwd)

sys.path.append(os.path.join(cwd, 'utils'))
sys.path.append(os.path.join(cwd, 'tcn_models'))

# pd.set_option('display.max_rows', None)
from utils import tools
from tcn_models import TCNMultiTarget

model_dir = os.path.join(cwd[:cwd.find('IFG_DFO/RL')], 'IFG_DFO/RL/models/ifg_v5/rl_v6_multi_trgts_tcn_model')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# retrieve merged data from ifg_v5
ifg_ver = 5
# print (os.listdir(os.path.join(tools.get_ifg_dir(ifg_ver), 'merged')))
filename = "merged/inner_hist_wip_merge_2sec.csv"

data_df = tools.get_ifg_file (ifg_ver, filename)
print(data_df.shape)

### -------------------------------------------------------------------
# Target Variables by Group
# -------------------------------------------------------------------
ctrl_vars = ['Dryer/process/hmi_pide_moisture_comp_cv',  # dryer_temp_cv
             'Dryer/vfds/hmi_vfd_01/status/act_freq',        # dryer_feed_rate
             'Dryer/vfds/hmi_vfd_03/status/act_freq']        # supply_feed_rate

hist_p1_vars = ['Dryer/data/hmi_mt_2_reading_1_real', 'Dryer/data/hmi_mt_2_temperature_1_real']

hist_p2_vars = ['Dryer/data/hmi_mt_1_reading_1_real', 'Dryer/data/hmi_mt_1_temperature_1_real']

hist_p3_vars = ['Dryer/data/vibe_temp_f', 'Dryer/data/vibe_exhaust_temp_f',]

hist_p4_vars = ['Dryer/process/hmi_pide_moisture_comp_pv', 'Dryer/process/hmi_pide_moisture_comp_sp',]
                
wip_p1_vars = ['Distance', 'Particles']

wip_p2_vars = ['X50', 'Xc', 'D01', 'D05', 'D10', 'D20', 'D25', 'D50', 'D75', 'D80', 'D90', 'D95', 'D99']

wip_p3_vars = ['3 mm', '3.5 mm', '4 mm', '5 mm', '6 mm', '7 mm', '8 mm', '9 mm', '10 mm', '11 mm', '12 mm', '13 mm', '14 mm']


### -------------------------------------------------------------------
# Variable Preparation
# -------------------------------------------------------------------
group_names = ['hist_p1', 'hist_p2', 'hist_p3', 'hist_p4', 'wip_p1', 'wip_p2', 'wip_p3']
target_vars_dict = {'hist_p1': hist_p1_vars, 'hist_p2': hist_p2_vars, 'hist_p3': hist_p3_vars, 'hist_p4': hist_p4_vars,
                    'wip_p1': wip_p1_vars, 'wip_p2': wip_p2_vars, 'wip_p3': wip_p3_vars,}

state_vars = [target_vars_dict[grp] for grp in group_names]
state_vars = list(chain.from_iterable(state_vars))
# state_vars

all_vars = ctrl_vars + state_vars
all_features = data_df[all_vars].to_numpy(dtype=np.float32)

/home/wensx/repos/IFG_DFO/RL/src/FY25_Oct/Shaw/rl_v6
(6039, 149)


In [28]:
group_idx = 0

target_vars = target_vars_dict[group_names[group_idx]] 
print(f"target group name: {group_names[group_idx]}\n{target_vars}")
targets = data_df[target_vars].to_numpy(dtype=np.float32)

target group name: hist_p1
['Dryer/data/hmi_mt_2_reading_1_real', 'Dryer/data/hmi_mt_2_temperature_1_real']


#### _Preprocess & Scaling_

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# -------------------------------------------------------------------
# Create Fixed-Length Sequences with a Sliding Window
# -------------------------------------------------------------------
def create_sequences(data, targets, seq_length):
    # data: shape (total_timesteps, num_features)
    # targets: shape (total_timesteps, num_targets)
    sequences = [data[i:i+seq_length].T for i in range(len(data) - seq_length)]
    labels = [targets[i+seq_length] for i in range(len(data) - seq_length)]
    return np.array(sequences), np.array(labels)

# -------------------------------------------------------------------
# MinMax Scaling Functions
# -------------------------------------------------------------------
def minmax_scale_features(x, scaler=None, feature_range=(0,1)):
    """
    Scales features using MinMax scaling.
    x: numpy array of shape (samples, num_features, seq_length)
    Returns: scaled x and the fitted scaler.
    """
    samples, num_features, seq_length = x.shape
    # Reshape to (samples*seq_length, num_features)
    x_reshaped = x.transpose(0, 2, 1).reshape(-1, num_features)
    if scaler is None:
        scaler = MinMaxScaler(feature_range=feature_range)
        x_scaled = scaler.fit_transform(x_reshaped)
    else:
        x_scaled = scaler.transform(x_reshaped)
    # Reshape back to (samples, seq_length, num_features) then transpose to (samples, num_features, seq_length)
    x_scaled = x_scaled.reshape(samples, seq_length, num_features).transpose(0, 2, 1)
    return x_scaled, scaler

def minmax_scale_targets(y, scaler=None, feature_range=(0,1)):
    """
    Scales targets using MinMax scaling.
    y: numpy array of shape (samples, num_targets)
    Returns: scaled y and the fitted scaler.
    """
    if scaler is None:
        scaler = MinMaxScaler(feature_range=feature_range)
        y_scaled = scaler.fit_transform(y)
    else:
        y_scaled = scaler.transform(y)
    return y_scaled, scaler

# -------------------------------------------------------------------
# Standard Scaling Functions
# -------------------------------------------------------------------
def standard_scale_features(x, scaler=None):
    """
    Scales features using Standard (z-score) scaling.
    x: numpy array of shape (samples, num_features, seq_length)
    Returns: scaled x and the fitted scaler.
    """
    samples, num_features, seq_length = x.shape
    # Reshape to (samples*seq_length, num_features)
    x_reshaped = x.transpose(0, 2, 1).reshape(-1, num_features)
    if scaler is None:
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x_reshaped)
    else:
        x_scaled = scaler.transform(x_reshaped)
    # Reshape back to (samples, seq_length, num_features) then transpose to (samples, num_features, seq_length)
    x_scaled = x_scaled.reshape(samples, seq_length, num_features).transpose(0, 2, 1)
    return x_scaled, scaler

def standard_scale_targets(y, scaler=None):
    """
    Scales targets using Standard (z-score) scaling.
    y: numpy array of shape (samples, num_targets)
    Returns: scaled y and the fitted scaler.
    """
    if scaler is None:
        scaler = StandardScaler()
        y_scaled = scaler.fit_transform(y)
    else:
        y_scaled = scaler.transform(y)
    return y_scaled, scaler

# -------------------------------------------------------------------
# Dataset for Multi-Target Training
# -------------------------------------------------------------------
class MultiTargetDataset(Dataset):
    def __init__(self, features, targets):
        """
        features: numpy array of shape (num_samples, total_vars, seq_length)
        targets: numpy array of shape (num_samples, num_state_vars)
        """
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [22]:
# -------------------------------------------------------------------
# Data Preparation
# -------------------------------------------------------------------
max_window = 510  # fixed sequence length (e.g., for 2sec intervals)

# Assume all_features and targets are defined elsewhere.
# all_features: numpy array of shape (total_timesteps, len(all_vars))
# targets: numpy array of shape (total_timesteps, num_state_vars)

# Create sequences:
# x_seq shape: (samples, len(all_vars), seq_length)
# y_seq shape: (samples, num_state_vars)
x_seq, y_seq = create_sequences(all_features, targets, max_window)

# Split data while preserving time order
x_train, x_val, y_train, y_val = train_test_split(
    x_seq, y_seq, test_size=0.1, shuffle=False, random_state=42
)

# # Apply MinMax scaling to the features and targets.
# # Note: We fit the scaler on the training data and then transform both training and validation sets.
# x_train_scaled, feat_scaler = minmax_scale_features(x_train)
# y_train_scaled, targ_scaler = minmax_scale_targets(y_train)
# x_val_scaled, _ = minmax_scale_features(x_val, scaler=feat_scaler)
# y_val_scaled, _ = minmax_scale_targets(y_val, scaler=targ_scaler)


x_train_scaled, feat_scaler = standard_scale_features(x_train)
y_train_scaled, targ_scaler = standard_scale_targets(y_train)
x_val_scaled, _ = standard_scale_features(x_val, scaler=feat_scaler)
y_val_scaled, _ = standard_scale_targets(y_val, scaler=targ_scaler)

# Create dataset objects using the scaled data
train_dataset = MultiTargetDataset(x_train_scaled, y_train_scaled)
val_dataset = MultiTargetDataset(x_val_scaled, y_val_scaled)

#### _Save Info for RL Training_

In [23]:
print("max_window:", max_window)

rl_setup_info = {
    'data': data_df,
    'ctrl_vars': ctrl_vars,
    'state_vars': state_vars,
    'group_name': group_names[group_idx],
    'trgt_vars': target_vars,
    'seq_len': max_window,
    'ftr_scaler': feat_scaler,
    'trgt_scaler': targ_scaler,
}

# save_dir = tools.get_ifg_dir(ifg_ver)
# save_dir = os.path.join(save_dir, 'train', 'v7')
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
#     assert os.path.isdir(save_dir), 'Save directory not created!'

# fn = f"{group_names[group_idx]}_tcn_seqlen{max_window}_inner_merge_2sec_standard.pkl"
# with open(os.path.join(save_dir, fn), 'wb') as file:
#     pickle.dump(rl_setup_info, file)
#     print(f"saved to: {os.path.join(save_dir, fn)}")

max_window: 510


### _Train Multi-Target TCN_

In [24]:
# ### -------------------------------------------------------------------
# # Training Loop for TCNMultiTarget Model
# # -------------------------------------------------------------------
# # weight_decay = 5e-3 -> too strong of a regularization
# # best at weight_decay = 1e-3
# def train_model(model, train_loader, val_loader, model_path, num_epochs=10, lr=1e-4, weight_decay=5e-4, device="cuda"):
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#     loss_fn = nn.MSELoss()
#     model.to(device)

#     best_train_loss = float('inf')
#     best_val_loss = float('inf')
#     best_overall_loss = float('inf')
#     best_model_state = None

#     for epoch in range(num_epochs):
#         is_best_train_loss = False
#         is_best_val_loss = False
        
#         model.train()
#         total_train_loss = 0.0
#         for batch in train_loader:
#             x, y = [b.to(device) for b in batch]  # x: (batch, len(all_vars), seq_length), y: (batch, num_state_vars)
#             optimizer.zero_grad()
#             outputs = model(x)
#             # Concatenate predictions from each target to form a tensor of shape (batch, num_state_vars)
#             preds = torch.cat([outputs[target] for target in model.target_vars], dim=1)
#             loss = loss_fn(preds, y)
#             loss.backward()
#             optimizer.step()
#             total_train_loss += loss.item()
#         avg_train_loss = total_train_loss / len(train_loader)
        
#         is_best_train_loss = avg_train_loss < best_train_loss
#         if is_best_train_loss:
#             best_train_loss = avg_train_loss

#         model.eval()
#         total_val_loss = 0.0
#         with torch.no_grad():
#             for batch in val_loader:
#                 x, y = [b.to(device) for b in batch]
#                 outputs = model(x)
#                 preds = torch.cat([outputs[target] for target in model.target_vars], dim=1)
#                 loss = loss_fn(preds, y)
#                 total_val_loss += loss.item()
#         avg_val_loss = total_val_loss / len(val_loader)
        
#         is_best_val_loss = avg_val_loss < best_val_loss
#         if is_best_val_loss:
#             best_val_loss = avg_val_loss  
        
#         print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")
#         overall_loss = avg_train_loss * 0.5 + avg_val_loss * 0.5
#         # if is_best_train_loss and is_best_val_loss:
#         if overall_loss < best_overall_loss:
#             best_overall_loss = overall_loss
#             best_model_state = model.state_dict()
#             print (f"**************** Flagged for Saving! ******************\n")

#     print("Training completed!")
#     torch.save(best_model_state, model_path)
#     print(f"Best model saved to: {model_path}")

# # -------------------------------------------------------------------
# # Instantiate and Train the Model
# # -------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define TCN hyperparameters
kernel_size = 3
num_layers = 5

model = TCNMultiTarget(
    all_vars=all_vars,
    target_vars=target_vars,
    kernel_size=kernel_size,
    num_layers=num_layers,
    num_channels=[32, 32, 32, 64, 64], # 128], # 128, 128], 
    custom_dilations=[8, 16, 33, 66, 132], # effective RF = 511
    # custom_dilations=[7, 14, 29, 58, 116], # RF = 449
    dropout=0.2,
)

# batch_size = 32
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
# val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# # Specify model save path (ensure model_dir is defined appropriately)
# model_path = os.path.join(model_dir, f'{group_names[group_idx]}_standard_rf{max_window}_state_dict.pt')

# train_model(model, train_loader, val_loader, model_path=model_path, 
#             num_epochs=150, lr=1.5e-4, weight_decay=5e-4, device=device)

# # min-max: train loss: 0.002-0.005, val loss: 0.003-0.007
# # standard-scaler: train: 0.05 - 0.2, val: 0.1-0.3
# print("Training finished!")

Using device: cuda
Epoch [3/150] - Train Loss: 0.4548 - Val Loss: 7.8315
Epoch [4/150] - Train Loss: 0.3165 - Val Loss: 6.0636
Epoch [5/150] - Train Loss: 0.2584 - Val Loss: 7.5321
Epoch [6/150] - Train Loss: 0.2259 - Val Loss: 6.8648
Epoch [7/150] - Train Loss: 0.2009 - Val Loss: 7.5067
Epoch [8/150] - Train Loss: 0.1882 - Val Loss: 7.3725
Epoch [9/150] - Train Loss: 0.1787 - Val Loss: 6.5456
Epoch [10/150] - Train Loss: 0.1653 - Val Loss: 5.7846
Epoch [11/150] - Train Loss: 0.1558 - Val Loss: 4.8715
Epoch [12/150] - Train Loss: 0.1522 - Val Loss: 3.7300
**************** Flagged for Saving! ******************
Epoch [13/150] - Train Loss: 0.1727 - Val Loss: 3.9377
Epoch [14/150] - Train Loss: 0.1891 - Val Loss: 4.1406
Epoch [15/150] - Train Loss: 0.1806 - Val Loss: 3.6850
**************** Flagged for Saving! ******************
Epoch [16/150] - Train Loss: 0.1892 - Val Loss: 2.8488
**************** Flagged for Saving! ******************
Epoch [17/150] - Train Loss: 0.2017 - Val Loss: 3.

### Evaluation

#### _Load TCNModel_

In [25]:
from collections import deque

state_dict_path = os.path.join(cwd[:cwd.find('IFG_DFO/RL')], 
                               'IFG_DFO/RL/models/ifg_v5/multi_trgt_tcn_models',
                               f'{group_names[group_idx]}_standard_rf510_state_dict.pt')

model_path = state_dict_path

model_state_dict = torch.load(model_path, weights_only=True)
model.load_state_dict(model_state_dict)

model.to(device)
model.eval()

train_data = rl_setup_info['data'][rl_setup_info['ctrl_vars']+rl_setup_info['state_vars']]

seq_len = rl_setup_info['seq_len']
buffer = deque(maxlen=seq_len)

for idx, row in train_data.iterrows():
    if idx < seq_len-1:
        buffer.append(row)
    else:
        break

preds_list = []
for idx, row in train_data[seq_len-1:].iterrows():
    buffer.append(row)
    # Build the TCN input from the buffer
    seq_data = np.array([row for row in buffer], dtype=np.float32)

    seq_data = np.expand_dims(seq_data.T, axis=0)
    # seq_data, _ = minmax_scale_features(seq_data, scaler=rl_setup_info['ftr_scaler'])
    seq_data, _ = standard_scale_features(seq_data, scaler=rl_setup_info['ftr_scaler'])    
    
    with torch.no_grad():
        inp_tensor = torch.tensor(seq_data, dtype=torch.float32, device=device)
        pred_dict = model(inp_tensor)
        
    # # Reassemble predictions into a vector following the order in state_vars.
    # var_names = []
    preds = []
    for var in rl_setup_info['trgt_vars']:
        # var_names.append(var)
        preds.append(pred_dict[var].item())
        
    preds = np.array(preds, dtype=np.float32)
    # Convert normalized value back to original scales.
    preds = rl_setup_info['trgt_scaler'].inverse_transform(preds.reshape(1, -1)).flatten()
    preds_list.append(preds)

In [26]:
print(f"target group name: {group_names[group_idx]}\n")

for idx, name in enumerate(target_vars):
    print(f"{name}")
    preds = [preds_tuple[idx] for preds_tuple in preds_list]
    for prct in [25, 50, 75, 95]:
        print(f"{prct}%: {np.percentile(preds, prct)}")
    print (f"")

target group name: hist_p3

Dryer/data/vibe_temp_f
25%: 72.17394828796387
50%: 136.8531036376953
75%: 171.08401107788086
95%: 207.22205352783203

Dryer/data/vibe_exhaust_temp_f
25%: 223.31259155273438
50%: 269.6506042480469
75%: 332.33924102783203
95%: 370.73231506347656

